In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
import pandas as pd

In [6]:
url = 'https://foursquare.com/explore?mode=url&ne=24.66137%2C46.765409&q=Food&sw=24.601685%2C46.668591'
page = requests.get(url)
print(page.status_code)

200


In [7]:
soup = BeautifulSoup(page.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [8]:
page_title = soup.title.text # gets you the text of the <title>(...)</title>
page_title

'Food Al-Margab'

In [9]:
driver = webdriver.Chrome()
# request the page 
driver.get(url)

        # extract the body and parse it
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

In [26]:

driver = webdriver.Chrome()
url = 'https://foursquare.com/explore?mode=url&ne=24.66137%2C46.765409&q=Food&sw=24.601685%2C46.668591'
columns_ = ['Name', 'Score', 'Category', 'Price_Range']
df = pd.DataFrame(columns=columns_)

for i in range(2, 12):  # Adjust the range to try 10 pages
    # Request the page
    driver.get(url)

    # Extract the body and parse it
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract names, scores, categories, and price ranges from the page
    names = soup.find_all('div', {'class': 'venueName'})
    scores = soup.find_all('div', {'class': 'venueScore positive', 'title': '', 'style': 'background-color: #00B551;'})
    categories = soup.find_all('span', {'class': 'categoryName'})
    price_ranges = soup.find_all('span', {'class': 'venueDataItem'})

    # Clean the data
    clean_Name = [p.text.strip() for p in names]
    clean_Score = [n.text.strip() for n in scores]
    clean_Category = [m.text.strip() for m in categories]

    # Extract and clean price ranges
    clean_Price_Range = []
    for span in price_ranges:
        price_span = span.find('span', {'class': 'price'})
        if price_span:
            price_text = price_span.find('span', {'class': 'darken'})
            if price_text:
                clean_Price_Range.append(price_text.text.strip())
            else:
                clean_Price_Range.append('Unknown')
        else:
            clean_Price_Range.append('Unknown')

    # Check if all lists have the same length before creating the DataFrame
    min_length = min(len(clean_Name), len(clean_Score), len(clean_Category), len(clean_Price_Range))
    clean_Name = clean_Name[:min_length]
    clean_Score = clean_Score[:min_length]
    clean_Category = clean_Category[:min_length]
    clean_Price_Range = clean_Price_Range[:min_length]
    
    # Store the data in a DataFrame
    new_rows_df = pd.DataFrame({
        'Name': clean_Name,
        'Score': clean_Score,
        'Category': clean_Category,
        'Price_Range': clean_Price_Range,
    })

    df = pd.concat([df, new_rows_df], ignore_index=True)
    
    url = f'https://foursquare.com/explore?mode=url&ne=24.66137%2C46.765409&q=Food&sw=24.601685%2C46.668591&offset={i}'
    
driver.quit()



In [27]:
df

,Name,Score,Category,Price_Range
0,1. فرن صحيفة وفطائر شامية,9.3,Bakery,Unknown
1,2. مطاعم ومطابخ باخلعه- مندي ومكتوم,9.2,Middle Eastern,$
2,3. محامص ومطاحن السوادي,9.2,Snacks,Unknown
3,4. Fawal Alafrah,9.1,Breakfast,$$
4,5. Finding Sushi,9.1,Sushi,Unknown
...,...,...,...,...
75,4. Fawal Alafrah,9.1,Breakfast,$$
76,5. Finding Sushi,9.1,Sushi,Unknown
77,6. AL TYSEER,9.0,Middle Eastern,Unknown
78,7. AlNaqaa,9.0,Breakfast,$
